In [7]:
import pandas as pd
import numpy as np

train_data = pd.read_csv('covid.train.csv')
test_data = pd.read_csv('covid.test.csv')
print(train_data.shape)
train_data.head()

(2699, 118)


,id,AL,AK,AZ,AR,CA,CO,CT,FL,GA,...,work_outside_home.4,shop.4,restaurant.4,spent_time.4,large_event.4,public_transit.4,anxious.4,depressed.4,worried_finances.4,tested_positive.4
0,0,0,0,0,0,0,0,0,1,0,...,31.113209,67.394551,36.674291,40.743132,17.842221,4.093712,10.440071,8.627117,37.329512,7.456154
1,1,0,0,0,0,0,1,0,0,0,...,33.920257,64.398380,34.612238,44.035688,17.808103,4.924935,10.172662,9.954333,32.508881,8.010957
2,2,0,0,0,0,0,0,0,0,0,...,31.604604,62.101064,26.521875,36.746453,13.903667,7.313833,10.388712,7.956139,36.745588,2.906977
3,3,0,0,0,0,0,0,0,0,0,...,35.115738,67.935520,38.022492,48.434809,27.134876,3.101904,10.498683,8.231522,38.680162,12.575816
4,4,0,0,0,0,0,0,0,0,0,...,35.129714,69.934592,38.242368,49.095933,22.683709,4.594620,9.878927,9.469290,28.344123,21.428589


In [8]:
test_data.head()

,id,AL,AK,AZ,AR,CA,CO,CT,FL,GA,...,travel_outside_state.4,work_outside_home.4,shop.4,restaurant.4,spent_time.4,large_event.4,public_transit.4,anxious.4,depressed.4,worried_finances.4
0,0,0,0,0,0,0,0,0,0,0,...,18.463415,32.470622,65.236954,31.730028,42.031183,18.188452,3.472105,12.452570,9.400024,30.550873
1,1,0,0,0,0,0,0,0,0,0,...,24.201702,31.499036,60.360030,28.319417,39.919723,16.614292,5.992332,11.747528,7.438606,35.364871
2,2,0,0,0,0,0,0,0,0,0,...,13.742657,28.696996,66.114428,31.211381,40.448538,16.998722,13.951631,12.898089,10.214711,37.028705
3,3,0,0,0,0,0,0,0,0,0,...,13.763442,27.907935,66.581525,29.112434,41.123571,13.949904,5.337427,14.944972,11.556292,32.146346
4,4,0,0,0,0,0,0,0,0,0,...,14.427636,37.623197,66.502657,29.716613,41.790256,18.855524,3.015219,15.225684,10.126752,39.163694


通过这两个的比较，我们知道，我们需要使用前117列数据来预测第118列的数据，还是欠缺了很多东西，这里简单跑一个线性回归
处理features和labels就行，然后跑一下test

In [9]:
features = np.array(train_data)[0:,:117]
labels = np.array(train_data)[0:,117:]
labels

array([[ 7.4561538],
       [ 8.010957 ],
       [ 2.9069774],
       ...,
       [13.6182563],
       [ 9.5125846],
       [11.0694227]])

In [10]:
from mxnet import ndarray as nd
features = nd.array(features)
labels = nd.array(labels)
labels = labels.reshape(shape=[2699])

ModuleNotFoundError: No module named 'mxnet'

In [ ]:
from mxnet.gluon import data as gdata

# 超参数
batch_size = 5
dataset = gdata.ArrayDataset(features,labels)
data_iter = gdata.DataLoader(dataset,batch_size,shuffle=True)

for X,y in data_iter:
    print(X,y)
    break

In [ ]:
from mxnet.gluon import nn
net = nn.Sequential()
net.add(nn.Dense(2000,activation='relu'),nn.Dense(1))

from mxnet import init
net.initialize(init.Normal(sigma=0.01))

from mxnet.gluon import loss as gloss

loss = gloss.L2Loss()

from mxnet import gluon
trainer = gluon.Trainer(net.collect_params(),'sgd',{'learning_rate':0.00000000001})

In [ ]:
from mxnet import autograd,nd
num_epochs = 1000
for epoch in range(1,num_epochs+1):
    for X,y in data_iter:
        with autograd.record():
            l = loss(net(X),y)
        l.backward()
        trainer.step(batch_size)
    l = loss(net(features),labels)
    print('epoch %d,loss: %f' % (epoch,l.mean().asnumpy()))

In [ ]:
test = nd.array(test_data)
y_hat = net(test)
y_hat.asnumpy()